In [1]:
import os
import time
import subprocess
from datetime import datetime
import pathlib

# 📂 Directorio actual (donde se ejecuta el script)
SOURCE_DIR = pathlib.Path("/home/jupyter/RedPitaya/DATOS").resolve()
# SOURCE_DIR = os.getcwd()
print(SOURCE_DIR)

/home/jupyter/RedPitaya/DATOS


In [2]:
# 🖥️ Carpeta compartida en Windows
DEST_IP = "10.73.28.39"
DEST_SHARE = "SHARED"
DEST_ROOT_SUBFOLDER = "Data"

# 👤 Credenciales SMB (usar "" si no necesita autenticación)
USER = "user"
PASSWORD = "tu_contraseña"

# ⏱️ Tiempo máximo sin archivos nuevos antes de terminar (segundos)
MAX_IDLE_TIME = 10 * 60  # 10 minutos

# 🧾 Archivo de log
LOG_FILE = SOURCE_DIR / "transfer_log.txt"

def log_message(msg):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_line = f"[{timestamp}] {msg}\n"
    print(log_line.strip())
    with open(LOG_FILE, "a") as log:
        log.write(log_line)

# 🧠 Verificar que smbclient esté disponible
if subprocess.run(["which", "smbclient"], stdout=subprocess.PIPE).returncode != 0:
    log_message("❌ smbclient no está instalado. Instálalo con: sudo apt install smbclient")
    exit()

# Preguntar al inicio si se borrarán los archivos locales después de la transferencia
borrar_local = input("🗑️ ¿Deseas borrar los archivos locales después de transferirlos? (s/N): ").strip().lower() == 's'

log_message(f"🔍 Monitoreando {SOURCE_DIR} para archivos HDF5 (incluye subdirectorios)...")

def is_file_in_use(filepath):
    """Verifica si un archivo está abierto con lsof."""
    try:
        result = subprocess.run(["lsof", str(filepath)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return bool(result.stdout.decode().strip())
    except Exception as e:
        log_message(f"⚠️ Error al ejecutar lsof: {e}")
        return False

def transfer_file(local_path, relative_path):
    """Transfiere un archivo manteniendo estructura de directorios."""
    dest_path = os.path.join(DEST_ROOT_SUBFOLDER, os.path.dirname(relative_path)).replace("\\", "/")
    smb_auth = f"-U {USER}%{PASSWORD}" if USER and PASSWORD else "-N"

    # Crear carpeta destino
    mkdir_cmd = f"smbclient //{DEST_IP}/{DEST_SHARE} {smb_auth} -c 'mkdir \"{dest_path}\"'"
    subprocess.run(mkdir_cmd, shell=True)

    # Comando de transferencia
    transfer_cmd = (
        f"smbclient //{DEST_IP}/{DEST_SHARE} {smb_auth} "
        f"-c 'cd \"{dest_path}\"; put \"{local_path}\" \"{os.path.basename(local_path)}\"'"
    )

    result = subprocess.run(transfer_cmd, shell=True)

    if result.returncode == 0:
        log_message(f"✅ Transferencia exitosa: {relative_path}")
        if borrar_local:
            os.remove(local_path)
            log_message(f"🗑️ Archivo eliminado localmente: {relative_path}")
        return True
    else:
        log_message(f"❌ Error al transferir: {relative_path}")
        return False

# 🔄 Bucle principal con tiempo de inactividad
last_new_file_time = datetime.now()

while True:
    new_file_found = False

    # Recorrer subdirectorios de SOURCE_DIR
    for subdir in [d for d in SOURCE_DIR.iterdir() if d.is_dir()]:
        h5_files = list(subdir.glob("*.h5"))
        if not h5_files:
            continue

        for file_path in h5_files:
            relative_path = file_path.relative_to(SOURCE_DIR)
            if is_file_in_use(file_path):
                log_message(f"⚠️ En uso, se reintentará luego: {relative_path}")
                continue

            if transfer_file(file_path, relative_path):
                new_file_found = True

        # Intentar borrar subdirectorio si queda vacío
        if borrar_local:
            try:
                subdir.rmdir()
                log_message(f"🗑️ Subdirectorio eliminado: {subdir}")
            except OSError:
                log_message(f"ℹ️ Subdirectorio '{subdir}' no está vacío, solo se eliminaron los archivos")

    # Verificar tiempo de inactividad
    if new_file_found:
        last_new_file_time = datetime.now()
    else:
        idle_time = (datetime.now() - last_new_file_time).total_seconds()
        if idle_time >= MAX_IDLE_TIME:
            log_message(f"⏱️ No se detectaron nuevos archivos en {MAX_IDLE_TIME / 60:.0f} minutos. Terminando programa.")
            break

    time.sleep(10)  # Revisa cada 10 segundos


🗑️ ¿Deseas borrar los archivos locales después de transferirlos? (s/N):  n


[2025-10-30 17:38:11] 🔍 Monitoreando /home/jupyter/RedPitaya/DATOS para archivos HDF5 (incluye subdirectorios)...


putting file /home/jupyter/RedPitaya/DATOS/Data_20251029_1717_TCH4_TL+00/Data_20251029_1717_0028.h5 as \Data\Data_20251029_1717_TCH4_TL+00\Data_20251029_1717_0028.h5 (6913.6 kb/s) (average 6913.6 kb/s)


[2025-10-30 17:38:14] ✅ Transferencia exitosa: Data_20251029_1717_TCH4_TL+00/Data_20251029_1717_0028.h5
NT_STATUS_OBJECT_NAME_COLLISION making remote directory \Data\Data_20251029_1717_TCH4_TL+00


putting file /home/jupyter/RedPitaya/DATOS/Data_20251029_1717_TCH4_TL+00/Data_20251029_1717_0023.h5 as \Data\Data_20251029_1717_TCH4_TL+00\Data_20251029_1717_0023.h5 (7603.6 kb/s) (average 7603.6 kb/s)


...

[2025-10-30 17:48:16] ✅ Transferencia exitosa: Data_20251029_1648_TCH3_TL+00/Data_20251029_1648_0002.h5
NT_STATUS_OBJECT_NAME_COLLISION making remote directory \Data\Data_20251029_1648_TCH3_TL+00


putting file /home/jupyter/RedPitaya/DATOS/Data_20251029_1648_TCH3_TL+00/Data_20251029_1648_0007.h5 as \Data\Data_20251029_1648_TCH3_TL+00\Data_20251029_1648_0007.h5 (7654.7 kb/s) (average 7654.7 kb/s)


[2025-10-30 17:48:19] ✅ Transferencia exitosa: Data_20251029_1648_TCH3_TL+00/Data_20251029_1648_0007.h5
NT_STATUS_OBJECT_NAME_COLLISION making remote directory \Data\Data_20251029_1648_TCH3_TL+00


putting file /home/jupyter/RedPitaya/DATOS/Data_20251029_1648_TCH3_TL+00/Data_20251029_1648_0010.h5 as \Data\Data_20251029_1648_TCH3_TL+00\Data_20251029_1648_0010.h5 (6884.2 kb/s) (average 6884.2 kb/s)


[2025-10-30 17:48:20] ✅ Transferencia exitosa: Data_20251029_1648_TCH3_TL+00/Data_20251029_1648_0010.h5
NT_STATUS_OBJECT_NAME_COLLISION making remote directory \Data\Data_20251029_1648_TCH3_TL+00
NT_STATUS_ACCESS_DENIED opening remote file \Data\Data_20251029_1648_TCH3_TL+00\Data_20251029_1648_0003.h5
[2025-10-30 17:48:21] ❌ Error al transferir: Data_20251029_1648_TCH3_TL+00/Data_20251029_1648_0003.h5
NT_STATUS_OBJECT_NAME_COLLISION making remote directory \Data\Data_20251029_1717_TCH4_TL+00
NT_STATUS_ACCESS_DENIED opening remote file \Data\Data_20251029_1717_TCH4_TL+00\Data_20251029_1717_0028.h5
[2025-10-30 17:48:32] ❌ Error al transferir: Data_20251029_1717_TCH4_TL+00/Data_20251029_1717_0028.h5
NT_STATUS_OBJECT_NAME_COLLISION making remote directory \Data\Data_20251029_1717_TCH4_TL+00
NT_STATUS_ACCESS_DENIED opening remote file \Data\Data_20251029_1717_TCH4_TL+00\Data_20251029_1717_0023.h5
[2025-10-30 17:48:33] ❌ Error al transferir: Data_20251029_1717_TCH4_TL+00/Data_20251029_1717_0